In [104]:
import bpy
import numpy as np

import os
from typing import List, Dict
from collections import deque
import bmesh

import scipy.sparse
import scipy.linalg
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

from line_profiler import profile

# install packages : "C:\Users\Pierre.Gilibert\OneDrive - ARVERNE\Bureau\blender-4.3.1-windows-x64\4.3\python\bin\python.exe" -m pip install scipy 

In [105]:
FOLDER = "C:\\Users\\pierr\\Documents\\Blender\\GeometryProcessing_Python\\"
# FOLDER = "C:\\Users\\Pierre.Gilibert\\OneDrive - ARVERNE\\Documents\\Divers\\blender"
bpy.ops.wm.open_mainfile(filepath=os.path.join(FOLDER, "blender_notebook_v01.blend"))

{'FINISHED'}

In [106]:


class HEEdge:
    def __init__(self, edge, face, orientation:int):
        self.edge = edge
        self.face = face
        self.orientation = orientation
        self.vertex = edge.verts[orientation] # orientation in {0,1} stating whether the self.edge is in the same (0) direction as self or the opposite (1) direction.
        self.next = None
        self.twin = None
        self.angle_from_X = 0.0
        self.transport_coeff = 0.0

    def __str__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"
    
    def __repr__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"

    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.edge, item)
    


In [107]:

class MYMesh:
    def __init__(self):
        self.bm = bmesh.new()
        self.facecorners = []
        self.vert2facecorner = {}
        self.facecorner_attributes = {}             # facecorners attributes
        self.vertex_attributes = {"u":{}, "v":{}}   # u,v : basis vector of the tangent plane at each vertex
        self.edge_attributes = {}
        self.face_attributes = {}
        self.face_vertex_to_facecorner = {}
        self.co = None                              # A (|V|, 3) np array where row i contains the x,y,z coordinates of the vertex indexed by i 
        self.cotan = None                           # A (3|F|,) np array where entry i is the cotangent of the angle of facecorner (loop) i
        self.face_areas = None                      # A (|F|,) np array where entry i is the area for face i
        self.internal_angles = None                 # A (3|F|,) np array where entry i is the angle of facecorner (loop) i
        self.fv = None                              # A (|F|, 3) np array where row i contains the indices of the vertices of face i
        self.heedges = []                           # List of Half edges
        self.dict_vert2heedges = {}


    def from_mesh(self, mesh_data):
        """ Mimic the bmesh from_mesh function. """
        self.bm.from_mesh(mesh_data)
        self.vert2facecorner = {v:[] for v in self.verts}
        self.facecorners = []
        for face in self.faces:
            self.face_vertex_to_facecorner[face] = {}
            for loop in face.loops:
                self.vert2facecorner[loop.vert].append(loop)
                self.facecorners.append(loop)
                self.face_vertex_to_facecorner[face][loop.vert] = loop
            
        self.fv = np.array([[fc.vert.index for fc in f.loops] for f in self.faces], dtype=int) # face #f has vertex [vi, vj, vk]
        self.co = np.array([v.co for v in self.verts])

    def to_mesh(self, mesh_data):
        """ Mimic the bmesh to_mesh function. """
        self.bm.to_mesh(mesh_data)
    
    # Add any other bmesh methods as needed:
    def free(self):
        """ Mimic the bmesh free function. """
        self.bm.free()

    # Add a custom method to directly access the internal bmesh:
    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.bm, item)
    
    def ensure_lookup_tables(self):
        self.verts.ensure_lookup_table()
        self.edges.ensure_lookup_table()
        self.faces.ensure_lookup_table()

    def create_halfedge_datastructure(self):
        self.dict_vert2heedges = {}
        # self.edge_attributes["heedges"] = {}
        # for e in self.edges:
        #     self.edge_attributes["heedges"][e] = []
        self.vertex_attributes["hedge"] = {} # vertex to hedge
        for f in self.faces:
            if len(f.verts) != 3:
                raise ValueError("Not a triangular mesh : triangulate it beforehand !!")
            v_orientation = [f.verts[0].index, f.verts[1].index, f.verts[2].index]
            face_heedges = []
            for e in f.edges:
                # e0, e1, e2 = f.edges[0], f.edges[1], f.edges[2]
                v0, v1 = v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index)
                orientation = 1 - ((abs(v0-v1) == 1 and v0 < v1) or (abs(v0-v1) == 2 and v0 > v1)) # 0 if the edge matches the ccw convention of the face, 1 otherwise
                # print(e, v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index), orientation)
                face_heedges.append(HEEdge(e, f, orientation))
            he0, he1, he2 = face_heedges
            he0.next = he1
            he1.next = he2
            he2.next = he0
            self.dict_vert2heedges[(he0.vertex.index, he1.vertex.index)] = he0
            self.dict_vert2heedges[(he1.vertex.index, he2.vertex.index)] = he1
            self.dict_vert2heedges[(he2.vertex.index, he0.vertex.index)] = he2
            self.heedges.append(he0)
            self.heedges.append(he1)
            self.heedges.append(he2)
            self.vertex_attributes["hedge"][he0.vertex] = he0
            self.vertex_attributes["hedge"][he1.vertex] = he1
            self.vertex_attributes["hedge"][he2.vertex] = he2
            # self.edge_attributes["heedges"][he0.edge] = he0
            # self.edge_attributes["heedges"][he1.edge] = he1
            # self.edge_attributes["heedges"][he2.edge] = he2

        for k in self.dict_vert2heedges:
            i0, i1 = k
            if i0 < i1:
                continue
            kr = (i1, i0)
            if kr in self.dict_vert2heedges:
                self.dict_vert2heedges[k].twin = self.dict_vert2heedges[kr]
                self.dict_vert2heedges[kr].twin = self.dict_vert2heedges[k]
        
        boundary_hedges = []
        for e in self.edges:
            if not e.is_boundary:
                continue
            i0, i1 = e.verts[0].index, e.verts[1].index
            if (i0, i1) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i0, i1)]
            elif (i1, i0) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i1, i0)]
            else:
                raise ValueError(f"Unable to find an half edge between vertices {i0} and {i1}")
            new_he = HEEdge(e, None, 1-existing_he.orientation)
            existing_he.twin = new_he
            new_he.twin = existing_he
            boundary_hedges.append(new_he)
            self.dict_vert2heedges[(new_he.vertex.index, e.other_vert(new_he.vertex).index)] = new_he
            self.heedges.append(new_he)

        # match the boundary hedges
        num_link_to_create = len(boundary_hedges)
        num_link_created = 0
        while num_link_created != num_link_to_create:
            he = boundary_hedges[num_link_created]
            # other_index = -1
            for i, ohe in enumerate(boundary_hedges):
                if ohe.vertex == he.twin.vertex:
                    other_index = i
                    break
            # ohe = boundary_hedges.pop(other_index)
            he.next = ohe
            num_link_created+=1


        # verification : all hedges should have a twin and a next defined
        for k in self.dict_vert2heedges:
            he = self.dict_vert2heedges[k]
            if he.twin is None or he.next is None:
                raise ValueError(f"Unable to define a twin or a next for hedge {k} : {he}")

    def _build_d0(self):

        row = np.repeat(np.arange(len(self.edges)), 2)
        col = np.array([y for e in self.edges for y in [e.verts[0].index, e.verts[1].index]])
        val = np.ones(len(col))
        val[1::2] = -1

        # matrix[e] = [0,0,..., 1, 0, ..., -1, ..., 0] +1 at vertex vi, -1 at vertex vj where e = [vi, vj] in that order
        return scipy.sparse.coo_matrix((val, (row, col)), shape=(len(self.edges), len(self.verts)))

    def _build_d1(self):
        edge_dict = {tuple(sorted([e.verts[0].index, e.verts[1].index])): i for i, e in enumerate(self.edges)}
        vi, vj, vk = self.fv[:,0], self.fv[:,1], self.fv[:,2]
        face_edges = np.reshape(np.vstack((vi, vj, vj, vk, vk, vi)).T, (len(self.faces), 3, 2))
        # face_edges[f] = [[i,j], [j,k], [k, i]] (shape 3,2) with i,j,k the vertex indices of face f and [i,j], [j,k] and [k,i] the 3 ordered edges of face f
        sorted_face_edges = np.sort(face_edges, axis=2)
        val = np.any(sorted_face_edges == face_edges, axis=2)*2-1 # +1 if edge in correct orientation, -1 otherwise
        col = np.array([edge_dict[tuple(e)] for e in sorted_face_edges.reshape((-1,2))])
        row = np.repeat(np.arange(len(self.faces)), 3)
        # matrix[f] = [0, 0,... , +-1, ..., +-1, ..., +-1, ..., 0] non 0 at ei, ej, ek where f.edges = [ei, ej, ek] and +-1 depending on whether the vertices of the ei, ej, ek are in the sorted order or not 
        return scipy.sparse.coo_matrix((val.ravel(), (row, col)), shape=(len(self.faces), len(self.edges)))

    def _build_hodge0(self, inverse=False):
        if not "area" in self.vertex_attributes:
            self._calculate_vertex_area()
        vertex_areas = self.vertex_attributes["area"]
        N = len(self.verts)
        row = np.arange(N)
        if inverse:
            return scipy.sparse.coo_matrix((1/vertex_areas, (row, row)), shape=(N, N))
        else:
            return scipy.sparse.coo_matrix((vertex_areas, (row, row)), shape=(N, N)) # dual / primal with primal = area of vertex = 1 by convention

    def _build_hodge1(self, inverse=False):
        edge_vertex = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_face_areas = np.zeros(len(self.edges))
        for e in self.edges:
            s = 0
            for f in e.link_faces:
                s += self.face_areas[f.index]
            edge_face_areas[e.index] = s

        edge_len = np.linalg.norm(self.co[edge_vertex[:,1]] - self.co[edge_vertex[:,0]], axis=1)
        # dual_edge_lengths = edge_face_areas / edge_len

        N = len(self.edges)
        row = np.arange(N)
        # return edge_len / edge_face_areas
        if inverse:
            return scipy.sparse.coo_matrix((edge_len / edge_face_areas, (row, row)), shape=(N, N))
        else:
            return scipy.sparse.coo_matrix((edge_face_areas / edge_len, (row, row)), shape=(N, N))

    def _build_hodge2(self, inverse=False):
        if not "area" in self.vertex_attributes:
            self._calculate_vertex_area()

        N = len(self.faces)
        row = np.arange(N)
        primal_areas = self.face_areas

        dual_areas = 1 #

        # fc_area = np.array([[self.facecorner_attributes["area"][self.face_vertex_to_facecorner[f][v].index] for f in self.faces for v in f.verts]]).reshape((-1,3))
        # weights = fc_area / self.face_areas[:,None]
        # vertex_area = self.vertex_attributes["area"][self.fv.ravel()].reshape((-1,3))
        # dual_areas = weights*vertex_area
        # dual_areas = dual_areas[:,0] + dual_areas[:,1] + dual_areas[:,2]
        if inverse:
            return scipy.sparse.coo_matrix((primal_areas/dual_areas, (row, row)), shape=(N, N))
        else:
            return scipy.sparse.coo_matrix((dual_areas/primal_areas, (row, row)), shape=(N, N))

    def construct_dec_operators(self, inverse_hodge0=False, inverse_hodge1=False, inverse_hodge2=False):
        """
        Constructs discrete exterior derivative operators (d0, d1) and
        Hodge star matrices (hodge0, hodge1, hodge2).
        """
        d0 = self._build_d0()
        d1 = self._build_d1()
        hodge0 = self._build_hodge0(inverse=inverse_hodge0)
        hodge1 = self._build_hodge1(inverse=inverse_hodge1)
        hodge2 = self._build_hodge2(inverse=inverse_hodge2)
        
        return d0, d1, hodge0, hodge1, hodge2

    def vector_field_to_1form(self, vector_field):
        """
        Projects a vertex-based tangent vector field onto a discrete 1-form on edges.

        Parameters:
        - vector_field: Nx3 numpy array, the tangent vector field at each vertex.

        Returns:
        - one_form: Mx1 numpy array, the discrete 1-form (edge-based representation).
        """
        
        edge_vertex = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_dir = self.co[edge_vertex[:,1]] - self.co[edge_vertex[:,0]]
        ui = vector_field[edge_vertex[:,0]]
        uj = vector_field[edge_vertex[:,1]]
        u_edge = 0.5 * (ui+uj)
        dot = np.einsum('ij,ij->i', u_edge, edge_dir) # fast dot product 
        return dot/np.linalg.norm(edge_dir, axis=1)
    
    def one_form_to_vector_field(self, one_form):
        """
        Maps a discrete 1-form (edge-based representation) back to a vertex-based tangent vector field.

        Parameters:
        - one_form: Mx1 numpy array, the discrete 1-form values on edges.

        Returns:
        - vector_field: Nx3 numpy array, the tangent vector field at each vertex.
        """

        edge_vertex = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_dir = self.co[edge_vertex[:,1]] - self.co[edge_vertex[:,0]]
        edge_dir = edge_dir / np.linalg.norm(edge_dir, axis=1)[:,None]
        contribution = one_form[:,None] * edge_dir

        row = np.repeat(np.arange(len(self.edges)),2).astype(int)
        col = edge_vertex.ravel().astype(int)
        val = np.ones(2 * len(self.edges))

        adjacency_matrix = scipy.sparse.coo_matrix((val, (row, col)))

        vertex_contributions = adjacency_matrix.T @ contribution  # Sum contributions per vertex
        vertex_edge_counts = adjacency_matrix.sum(axis=0).A1  # Number of edges per vertex (1D array)

        return vertex_contributions / vertex_edge_counts[:,None]

    
    def _calculate_corner_angles_and_face_areas(self):
        self.co = np.array([v.co for v in self.verts])

        vi = self.co[self.fv[:,0]]
        vj = self.co[self.fv[:,1]]
        vk = self.co[self.fv[:,2]]

        eij, ejk, eki = vj-vi, vk-vj, vi-vk
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2]
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]
        lij, ljk, lki = np.sqrt(lij2), np.sqrt(ljk2), np.sqrt(lki2)

        s = 0.5 * (lij + ljk + lki) # half perimeter of every triangle
        self.face_areas = np.sqrt(s * (s - lij) * (s - ljk) * (s - lki)) # Heron's formula for the area of the triangles
        
        q_i = -ljk2 + lij2 + lki2
        q_j = -lki2 + ljk2 + lij2
        q_k = -lij2 + lki2 + ljk2

        denom_inv = 1/(4*self.face_areas)
        self.cotan = np.zeros(3*len(bm.faces))
        self.cotan[0::3] = q_i*denom_inv
        self.cotan[1::3] = q_j*denom_inv
        self.cotan[2::3] = q_k*denom_inv

        self.internal_angles = np.zeros(3*len(bm.faces))
        self.internal_angles[0::3] = np.arccos(np.clip(q_i / (2*lij * lki), -1, 1))
        self.internal_angles[1::3] = np.arccos(np.clip(q_j / (2*ljk * lij), -1, 1))
        self.internal_angles[2::3] = np.arccos(np.clip(q_k / (2*lki * ljk), -1, 1))

    def calculate_angle_defect(self):
        angle_defects = []
        boundary_verts = []
        for v in self.verts:
            s = 0
            for fc in self.vert2facecorner[v]:
                s += self.internal_angles[fc.index]
            angle_defects.append(s)
            boundary_verts.append( 2 - v.is_boundary) # 2 if interior, 1 if bondary

        self.vertex_attributes["angle_defect"] = np.pi * np.array(boundary_verts) - np.array(angle_defects)
        
    def euler_characteristic(self):
        return len(self.verts) - len(self.edges) + len(self.faces)


In [108]:
obj = bpy.data.objects["Suzanne"]
# obj = bpy.data.objects["Torus.001"]

bpy.ops.object.mode_set(mode='OBJECT')
mesh = obj.data
bm = MYMesh()
bm.from_mesh(mesh)
bm.ensure_lookup_tables()
bm.create_halfedge_datastructure()

bm._calculate_corner_angles_and_face_areas()
bm.calculate_angle_defect()


In [112]:
class TransportData():
    def __init__(self):
        self.delta = None
        self.sign = None
        self.omega = None
        self.alphaI = None
        self.alphaJ = None

def calculate_star(self:MYMesh, e):
    # compute the dual/primal length ratio by adding up the
    # cotangents of the two angles opposing the current edge
    he:HEEdge
    sum = 0
    vi, vj = e.verts[0].index, e.verts[1].index
    he = self.dict_vert2heedges[(e.verts[0].index, e.verts[1].index)]
    keep_looping = True
    while keep_looping:
        vi = he.vertex.co
        vj = he.next.vertex.co
        vk = he.next.next.vertex.co
        # compute the cotangent of the angle of the current
        # triangle that opposes the current edge
        u = vi - vk
        v = vj - vk
        cot_theta = np.linalg.norm(np.dot(u,v)/np.cross(u,v))
        sum += 0.5 * cot_theta
        he = he.twin
        keep_looping = he != self.dict_vert2heedges[(e.verts[0].index, e.verts[1].index)]
        return max(sum, 0)


def geometry_change(self:MYMesh):
    self.edge_attributes["star"] = np.array([calculate_star(self, e) for e in self.edges])


def build_primal_spanning_tree(self:MYMesh):
    """
    Build a spanning tree of primal edges
    """
    # initialise the parent of each vertex to None : it means the vertex has not been visited yet.
    self.vertex_attributes["parent"] = {v:v for v in self.verts}
    has_been_seen = np.zeros(len(self.verts), dtype=bool)
    while not np.all(has_been_seen): # in practice : this loops over mesh islands
        root = None
        for v in self.verts:
            if not v.is_boundary and not has_been_seen[v.index]: # root is an unvisited non boundary vertex
                root = v
                break
        if root is None:
            raise ValueError("Unable to find a non-boundary vertex")
        
        queue = deque()
        queue.append(root)
        he:HEEdge

        while len(queue)>0:
            v = queue.popleft()
            has_been_seen[v.index] = True
            he = self.vertex_attributes["hedge"][v]
            keep_looping = True
            counter = 0
            while keep_looping and counter < len(v.link_edges) + 1:
                w = he.twin.vertex
                has_been_seen[w.index] = True # set to true even if it is on a boundary
                not_visited = self.vertex_attributes["parent"][w] == w
                if not_visited and w != root and not w.is_boundary:
                    self.vertex_attributes["parent"][w] = v
                    queue.append(w)
                he = he.twin.next
                keep_looping = he != self.vertex_attributes["hedge"][v]
                counter += 1
                if counter == len(v.link_edges)+1:
                    raise ValueError(f"Infinite loop detected : {counter} / {len(v.link_edges)+1}")

def in_primal_spanning_tree(self:MYMesh, he:HEEdge):
    v = he.vertex
    w = he.twin.vertex
    return self.vertex_attributes["parent"][v] == w or self.vertex_attributes["parent"][w] == v
    

def in_dual_spanning_tree(self:MYMesh, he:HEEdge):
    f = he.face
    g = he.twin.face
    return self.face_attributes["parent"][g] == f or self.face_attributes["parent"][f] == g

def build_dual_spanning_cotree(self:MYMesh):
    """
    builds a spanning tree of dual edges that do not cross the primal tree
    """
    def is_boundary(face):
        return face.edges[0].is_boundary or face.edges[1].is_boundary or face.edges[2].is_boundary
    
    self.face_attributes["parent"] = {f:f for f in self.faces}
    has_been_seen = np.zeros(len(self.faces), dtype=bool)  

    while not np.all(has_been_seen): # in practice : this loops over mesh islands
        root = None
        for f in self.faces:
            if is_boundary(f) or has_been_seen[f.index]:
                continue
            root = f # take a face that is not boundary and unvisited so far
            break
        

        queue = deque()
        queue.append(root)
        he:HEEdge
        while len(queue)>0:
            f = queue.popleft()
            has_been_seen[f.index] = True
            vi, vj = f.verts[0].index, f.verts[1].index
            he = self.dict_vert2heedges[(vi, vj)]
            keep_looping = True
            while keep_looping:
                g = he.twin.face
                if g is not None:
                    has_been_seen[g.index] = True
                    if self.face_attributes["parent"][g] == g and g != root and not in_primal_spanning_tree(self, he):
                        self.face_attributes["parent"][g] = f
                        queue.append(g)
                he = he.next
                keep_looping = he != self.dict_vert2heedges[(vi, vj)]


def build_tree_cotree_decomposition(self:MYMesh):
    
    build_primal_spanning_tree(self)
    build_dual_spanning_cotree(self)


def n_generators(self:MYMesh):
    """returns the number of independent noncontractible loops"""
    n = 0
    for e in self.edges:
        if e.is_boundary:
            continue

        vi, vj = e.verts[0], e.verts[1]
        he = self.dict_vert2heedges[(vi.index, vj.index)]
        if not in_primal_spanning_tree(self, he) and not in_dual_spanning_tree(self, he):
            n+=1
    return n

def get_shared_halfedge(self:MYMesh, faceA, faceB):
    vi, vj = faceA.verts[0].index, faceA.verts[1].index
    he = self.dict_vert2heedges[(vi,vj)]
    keep_looping = True
    counter = 0
    while keep_looping and counter < 1 + len(faceA.edges):
        if he.twin.face == faceB:
            return he
        he = he.next
        counter += 1
    raise ValueError(f"Unable to find a common (he)edge between face {faceA.index} and face {faceB.index}")
    
def get_dual_cycles(self:MYMesh):

    def get_half_cycle(f):
        hcycle = []
        counter = 0
        while counter < len(self.faces)+1:
            f_parent = self.face_attributes['parent'][f]
            if f == f_parent:
                break
            hcycle.append(get_shared_halfedge(self, f, f_parent))
            f = f_parent
            counter += 1
        return hcycle
    
    cycles = []
    for e in self.edges:
        if e.is_boundary:
            continue
        vi, vj = e.verts[0], e.verts[1]
        he = self.dict_vert2heedges[(vi.index, vj.index)]
        if not in_primal_spanning_tree(self, he) and not in_dual_spanning_tree(self, he):
            g = [he]
            c1 = get_half_cycle(he.twin.face)
            c2 = get_half_cycle(he.face)
            # print(len(c1), len(c2))
            # remove common hedge:
            m,n = len(c1)-1, len(c2)-1
            while c1[m] == c2[n]:
                n -= 1
                m -= 1
            # add them to the generator in the correct orientation
            for i in range(0, m+1):
                g.append(c1[i])
            for i in range(n, -1, -1):
                g.append(c2[i].twin)

            # remove hedges having a dangling vertex 
            vertex_occurence = {}
            for he in g:
                vertex_occurence[he.vertex] = 0
                vertex_occurence[he.twin.vertex] = 0
            for he in g:
                vertex_occurence[he.vertex] += 1
                vertex_occurence[he.twin.vertex] += 1
            to_keep = []
            for he in g:
                ocs, oce = vertex_occurence[he.vertex], vertex_occurence[he.twin.vertex]
                if min(ocs, oce) > 1: # not dangling
                    to_keep.append(he)
            # reorder the hedges:
            for i in range(len(to_keep)-1):
                heA, heB = to_keep[i], to_keep[i+1]
                if heA.vertex == heB.vertex:
                    to_keep[i] = heA.twin
            if is_boundary_loop(to_keep):
                to_keep = clean_boundary_loop(to_keep)

            g = to_keep
            cycles.append(g)
    return cycles

def clean_boundary_loop(cycle):
    he:HEEdge
    he = cycle[0]
    # Find an he on the boundary
    if not he.vertex.is_boundary:
        he = he.twin
    while he.face is not None:
        he = he.twin.next
    # loop over the boundary until coming back
    v0 = he.vertex
    cycle = []
    keep_looping = True    
    while keep_looping:
        cycle.append(he)
        he = he.next
        keep_looping = he.vertex != v0
    return cycle




def get_primal_cycles(self:MYMesh):
    cycles = []
    vertex2row = np.zeros(len(self.verts), dtype=int)
    for v in self.verts:
        if v.is_boundary:
            vertex2row[v.index] = -1
            cycles.append([])
            continue
        cycle = []
        he = self.vertex_attributes["hedge"][v]
        keep_looping = True
        while keep_looping:
            cycle.append(he)
            he = he.twin.next
            keep_looping = he != self.vertex_attributes["hedge"][v]
        vertex2row[v.index] = len(cycles)
        cycles.append(cycle)

    return cycles, vertex2row


def read_singularities_and_directional_constraints(self:MYMesh):
    self.face_attributes["constraint_angle"] = -np.ones(len(self.faces))
    self.vertex_attributes["singularities"] = np.zeros(len(self.verts))
    self.vertex_attributes["generators"] = np.zeros(len(self.verts))

def compute_face_frame(self:MYMesh):
    vi, vj, vk = self.co[self.fv[:,0]], self.co[self.fv[:,1]], self.co[self.fv[:,2]]
    e1 = vj - vi
    e1 = e1/np.linalg.norm(e1, axis=1)[:,None]
    e2 = vk - vi
    e2 = e2 - np.einsum('ij,ij->i', e1, e2)[:,None] * e1
    e2 = e2/np.linalg.norm(e2, axis=1)[:,None]
    self.face_attributes["e1"] = e1
    self.face_attributes["e2"] = e2 

def parallel_transport(self:MYMesh, phi:float, he:HEEdge):
    """
    Given an angle phi relative to the canonical reference frame of he.face, returns the angle parallel transported across he
    using the Levi-Civita connection, expressed relative to the canonical frame of he.twin.face
    """
      # Get (oriented) direction along shared edge
    u, v = he.vertex, he.twin.vertex
    e = v.co - u.co
    if u.index > v.index:
        e = -e

    # compute angle adjustments between canonical frames
    e1, e2 = self.face_attributes["e1"][he.face.index], self.face_attributes["e2"][he.face.index]
    f1, f2 = self.face_attributes["e1"][he.twin.face.index], self.face_attributes["e2"][he.twin.face.index]

    ee1 = e[0] * e1[0] + e[1] * e1[1] + e[2] * e1[2]
    ee2 = e[0] * e2[0] + e[1] * e2[1] + e[2] * e2[2]
    ef1 = e[0] * f1[0] + e[1] * f1[1] + e[2] * f1[2]
    ef2 = e[0] * f2[0] + e[1] * f2[1] + e[2] * f2[2]
    delta_ij = np.arctan2(ee2, ee1)
    delta_ji = np.arctan2(ef2, ef1)
    # Transport phi
    return phi - delta_ij + delta_ji

def get_directional_constraints(self:MYMesh):
    
    self.face_attributes["c_parent"] = {f:f for f in self.faces}
    has_been_seen = np.zeros(len(self.faces), dtype=bool)  

    transport_root = self.faces[0]
    for f in self.faces:
        if self.face_attributes["contraint_angle"] >= 0.0 :
            transport_root = f
            break
    
    queue = deque()
    queue.append(transport_root)
    he:HEEdge
    while len(queue)>0:
        f = queue.popleft()
        has_been_seen[f.index] = True
        vi, vj = f.verts[0].index, f.verts[1].index
        he = self.dict_vert2heedges[(vi, vj)]
        keep_looping = True
        while keep_looping:
            g = he.twin.face
            if g is not None:
                has_been_seen[g.index] = True
                if self.face_attributes["c_parent"][g] == g and g != transport_root:
                    self.face_attributes["c_parent"][g] = f
                    queue.append(g)

                    td = TransportData()
                    td.delta = parallel_transport(self, 0, he)
                    td.sign = 1
                    if he.vertex.index > he.twin.vertex.index:
                        sign = -1
                    raise NotImplementedError("TO DO")                    


def boundary_loop_curvature(self:MYMesh, cycle):
    totalK = 0
    he:HEEdge
    # Define the barycenter of the empty loop:
    barycentre = np.zeros(3)
    for he in cycle:
        barycentre += he.vertex.co
    barycentre = barycentre / len(cycle)

    K = 2*np.pi
    for he in cycle:
        vi, vj = np.array(he.vertex.co), np.array(he.twin.vertex.co)
        K -= tip_angle(barycentre, vi, vj)

      # add the curvature around each of the boundary vertices, using
      # the following labels:
      #    barycentre - virtual center vertex of boundary loop (computed above)
      #    d - current boundary vertex (we walk around the 1-ring of this vertex)
      #    a,b - consecutive interior vertices in 1-ring of d
      #    e,f - boundary vertices adjacent to d
    totalK += K

    for he in cycle:
        d = np.array(he.vertex.co)
        K = 2 * np.pi
        he2:HEEdge
        he2 = self.vertex_attributes["hedge"][he.vertex]
        keep_looping = True
        while keep_looping:
            if he2.face is None: #on boundary
                f = np.array(he2.next.vertex.co)
                K-=tip_angle(d, f, barycentre)
            else:
                a = np.array(he2.next.vertex.co)
                b = np.array(he2.next.next.vertex.co)
                # ////////////////// TO CHECK AT SOME POINT IN THE FUTURE (cf comment below) ///////////////////////////
                K-= tip_angle(d,a,b) # should be the same as K-= self.internal_angle(self.facevertextofacecorner[he2.face][he2.vertex]])
                if he2.twin.face is None:
                    e = np.array(he2.twin.vertex.co)
                    K -= tip_angle(d, barycentre, e)
            he2 = he2.twin.next
            keep_looping = he2 != self.vertex_attributes["hedge"][he.vertex]
        totalK += K

    return totalK
    


def cycle_defect(self:MYMesh, cycle):
    he:HEEdge
    theta = 0
    for he in cycle:
        theta = parallel_transport(self, theta, he)
    while theta >= np.pi : theta -= 2*np.pi
    while theta < - np.pi : theta += 2*np.pi
    return theta

def is_boundary_loop(cycle):
    if len(cycle) == 0:
        return False
    he:HEEdge
    he = cycle[0]
    return he.vertex.is_boundary or he.twin.vertex.is_boundary

def tip_angle(x, a, b):
    # returns the angle between a-x and b-x:
    u = a-x
    v = b-x
    u = u/np.linalg.norm(u)
    v = v/np.linalg.norm(v)
    xprod = np.cross(u,v)
    dot_prod = u[0]*v[0] + u[1]*v[1] + u[2]*v[2]
    return np.arctan2(np.linalg.norm(xprod), dot_prod)

read_singularities_and_directional_constraints(bm)
compute_face_frame(bm)
geometry_change(bm)
build_tree_cotree_decomposition(bm)
print(f"n_generators = {n_generators(bm)}")
dual_cycles = get_dual_cycles(bm)
primal_cycles, vertex2row = get_primal_cycles(bm)

n_generators = 4


In [127]:
class TrivialConnection():
    def __init__(self, mesh:MYMesh):
        self.mesh = mesh
        self.K = None
        self.b = None
        self.A = None
        self.primal_cycles, self.dual_cycles = None, None
        self.vertex2row = None
        
    def build_cycle_matrix(self, cycles):
        row, col, val = [], [], []
        he:HEEdge
        sqrt_star = np.sqrt(self.mesh.edge_attributes["star"])
        for l, c in enumerate(cycles):
            for he in c:
                k = he.edge.index
                i = he.vertex.index
                j = he.twin.vertex.index
                
                v = sqrt_star[k]
                if i > j:
                    v = - sqrt_star[k]

                row.append(k)
                col.append(l)
                val.append(v)

        self.A = scipy.sparse.coo_array((val, (row, col))).tocsr()
        
        

    def build(self):
        build_tree_cotree_decomposition(self.mesh)
        self.primal_cycles, self.vertex2row = get_primal_cycles(self.mesh)
        n_contractible_cycles = len(self.primal_cycles)

        self.dual_cycles = get_dual_cycles(self.mesh)
        cycles = self.primal_cycles + self.dual_cycles # concatenate all cycles
        n_basis_cycles = len(self.primal_cycles) + len(self.dual_cycles)
        A = self.build_cycle_matrix(cycles)
        K, generator_on_boundary = [], []
        for i in range(n_contractible_cycles):
            K.append(- self.mesh.vertex_attributes["angle_defect"][i])

        for i in range(n_contractible_cycles, n_contractible_cycles+len(self.dual_cycles)):
            if is_boundary_loop(cycles[i]):
                K.append(-boundary_loop_curvature(self.mesh, cycles[i]))
                generator_on_boundary.append(True)
            else:
                K.append(- cycle_defect(self.mesh, cycles[i]))
                generator_on_boundary.append(False)

        self.K = np.array(K)

    def setup_RHS(self):
        index_sum = 0
        self.b = self.K.copy()

        for v in self.mesh.verts:
            i = self.vertex2row[v.index]
            if i < 0:
                continue
            self.b[i] = self.K[i] + 2*np.pi * self.mesh.vertex_attributes["singularities"][v.index]
            index_sum += self.mesh.vertex_attributes["singularities"][v.index]

        n_contractible_cycles = len(self.primal_cycles)
        
        for j in range(n_contractible_cycles, n_contractible_cycles+len(self.dual_cycles)):
            i = j - n_contractible_cycles

            g=0 # to do
            self.b[j] = self.K[j] + g
            index_sum += g
        if( abs( index_sum-self.mesh.euler_characteristic() ) > 1e-7 ):
            print("/////////////////////////////////")
            print(f"Warning : indices do not add up to Euler characteristic!\n(solution may have unwanted singularities)\n\nEuler characteristic : {self.mesh.euler_characteristic()}\nsum of indices : {index_sum}")
            print("/////////////////////////////////")

    def solve(self):
        At = self.A.T
        AtA = At @ self.A
        AtA = AtA + scipy.sparse.eye(AtA.shape[0])*1e-9

        y = spsolve(AtA, self.b)
        x = self.A @ y
        print(x)


        
tc = TrivialConnection(bm)
tc.build()
tc.setup_RHS()
tc.solve()

/////////////////////////////////
(solution may have unwanted singularities)

Euler characteristic : 2
sum of indices : 0.0
/////////////////////////////////
[ 0.04330838  0.03804298  0.03609478 ... -0.07527267 -0.04354693
 -0.04615239]


In [95]:
len(bm.vertex_attributes["angle_defect"])

7958

In [76]:

# Create a new mesh and object
mesh_new = bpy.data.meshes.new("MyMesh")
obj_new = bpy.data.objects.new(f"{obj.name}_primal_tree", mesh_new)
bpy.context.collection.objects.link(obj_new)

tmp_mesh = bmesh.new()

for v in bm.verts:
    tmp_mesh.verts.new(v.co)
tmp_mesh.verts.ensure_lookup_table()

for v,w in bm.vertex_attributes["parent"].items():
    if v == w:
        continue
    vn, wn = tmp_mesh.verts[v.index],tmp_mesh.verts[w.index]
    tmp_mesh.edges.new((vn,wn))

tmp_mesh.to_mesh(mesh_new)


# Create a new mesh and object
mesh_new = bpy.data.meshes.new("MyMesh")
obj_new = bpy.data.objects.new(f"{obj.name}_dual_tree", mesh_new)
bpy.context.collection.objects.link(obj_new)
tmp_mesh = bmesh.new()
dict_tmp = {}
for f in bm.faces:
    vi, vj, vk = f.verts[0], f.verts[1], f.verts[2]
    barycentre = (vi.co + vj.co + vk.co)/3
    tmp_mesh.verts.new(barycentre)
    dict_tmp[f.index] = barycentre
tmp_mesh.verts.ensure_lookup_table()

for f,g in bm.face_attributes["parent"].items():
    if f == g:
        continue
    # v, w = dict_tmp[f], dict_tmp[g]
    v, w = tmp_mesh.verts[f.index],tmp_mesh.verts[g.index]
    tmp_mesh.edges.new((v,w))

    # vi, vj, vk = g.verts[0], g.verts[1], g.verts[2]
    # vi, vj, vk = tmp_mesh.verts[vi.index],tmp_mesh.verts[vj.index], tmp_mesh.verts[vk.index]
    # try:
    #     tmp_mesh.faces.new((vi, vj, vk))
    # except ValueError:
    #     pass

tmp_mesh.to_mesh(mesh_new)

dual_cycles = get_dual_cycles(bm)

mesh_new = bpy.data.meshes.new("MyMesh")
obj_new = bpy.data.objects.new(f"{obj.name}_dual_generators", mesh_new)
bpy.context.collection.objects.link(obj_new)
tmp_mesh = bmesh.new()

for v in bm.verts:
    tmp_mesh.verts.new(v.co)
tmp_mesh.verts.ensure_lookup_table()
for cycle in dual_cycles:
    for he in cycle:
        vi, vj = he.verts[0], he.verts[1]
        vi, vj = tmp_mesh.verts[vi.index], tmp_mesh.verts[vj.index]
        try:
            tmp_mesh.edges.new((vi, vj))
        except ValueError:
            pass
    
tmp_mesh.to_mesh(mesh_new)



In [90]:
tmp_mesh.verts[0].co

Vector((1.2377736568450928, -0.008821439929306507, -0.006944444961845875))

In [66]:
for v,w in bm.vertex_attributes["parent"].items():
    if v == w:
        print(v)

<BMVert(0x000001AF2C044290), index=0>
<BMVert(0x000001AF2C046EF8), index=203>
<BMVert(0x000001AF067A32D8), index=1919>
<BMVert(0x000001AF2BA78FD8), index=3071>


In [20]:
obj.name

'Torus.001'